In [ ]:
!pip install --quiet stanza transformers datasets tokenizers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [1]:
!unzip -q lessons.zip -d ./lessons


In [ ]:
# !pip install --quiet stanza transformers datasets tokenizers torch

import os, pickle, random
import torch
from torch import nn
from datasets import Dataset
from transformers import (
    GPT2Config, GPT2TokenizerFast,
    Trainer, TrainingArguments, GPT2LMHeadModel
)
from sklearn.model_selection import train_test_split

# STEP 1 – Setup
import stanza
stanza.download("en", verbose=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MAX_COMBOS = 1024
combo2id = {"<UNK>": 0}
_next_id = 1
def dynamic_get_combo_id(pos, dep, ner, feats):
    global _next_id
    key = (pos, dep, ner, feats)
    if key not in combo2id and _next_id < MAX_COMBOS:
        combo2id[key] = _next_id
        _next_id += 1
    return combo2id.get(key, 0)

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

import re

# STEP 2 – Load pre-parsed lessons
pkl_folder = "./lessons/lessons"
lessons = {}

def extract_vol_lesson(filename):
    match = re.match(r'v(\d+)_l(\d+)\.pkl', filename)
    return (int(match.group(1)), int(match.group(2))) if match else (999, 999)

all_files = os.listdir(pkl_folder)
pkl_files = sorted(
    [f for f in all_files if re.match(r'v\d+_l\d+\.pkl', f)],
    key=extract_vol_lesson
)

for fname in pkl_files:
    with open(os.path.join(pkl_folder, fname), "rb") as f:
        lessons[fname] = pickle.load(f)
    print(f"✅ Loaded {fname} ({len(lessons[fname])} docs)")


# STEP 3 – Combo extraction and encoding
def scan_doc(doc):
    ents = [(e.start_char, e.end_char, e.type) for e in doc.ents]
    for w in [w for s in doc.sentences for w in s.words]:
        ner = next((t for s0, e0, t in ents if s0 <= w.start_char < e0), "O")
        dynamic_get_combo_id(w.upos, w.deprel, ner, w.feats)

def encode_doc(doc):
    text = doc.text
    enc = tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)
    ents = [(e.start_char, e.end_char, e.type) for e in doc.ents]
    words = [w for s in doc.sentences for w in s.words]

    combo_ids = []
    for start, end in enc["offset_mapping"]:
        if start == end or end <= start:
            combo_ids.append(0)
            continue

        match = None
        for w in words:
            # allow small overlap fuzziness (fixes edge mismatches)
            if not (w.end_char <= start or w.start_char >= end):
                match = w
                break

        if match:
            ner = next((t for s0, e0, t in ents if s0 <= match.start_char < e0), "O")
            combo_ids.append(dynamic_get_combo_id(match.upos, match.deprel, ner, match.feats))
        else:
            combo_ids.append(0)
    num_total, num_missed = 0, 0
    num_total += len(enc["offset_mapping"])
    num_missed += sum(1 for c in combo_ids if c == 0)

    # print(f"💡 Miss rate: {100 * num_missed / num_total:.2f}%")

    return enc["input_ids"], combo_ids

def chunkify_docs(docs, max_len=64):
    """Each doc becomes one padded (or truncated) sample."""
    samples = []
    for doc in docs:
        ids, cmb = encode_doc(doc)
        ids = ids[:max_len]
        cmb = cmb[:max_len]
        pad = max_len - len(ids)
        if pad:
            ids += [tokenizer.pad_token_id] * pad
            cmb += [0] * pad
        samples.append({"input_ids": ids, "combo_ids": cmb})
    return Dataset.from_list(samples)


def collate(batch, max_len=64):
    ids, cmb = [], []
    for b in batch:
        input_ids = b["input_ids"][:max_len]
        combo_ids = b["combo_ids"][:max_len]
        pad = max_len - len(input_ids)
        input_ids += [tokenizer.pad_token_id] * pad
        combo_ids += [0] * pad
        ids.append(input_ids)
        cmb.append(combo_ids)

    ids = torch.tensor(ids)
    cmb = torch.tensor(cmb)
    mask = (ids != tokenizer.pad_token_id).long()

    # Mask out loss for padding
    labels = ids.clone()
    labels[ids == tokenizer.pad_token_id] = -100

    return {
        "input_ids": ids,
        "combo_ids": cmb,
        "attention_mask": mask,
        "labels": labels,
    }

# STEP 4 – Model
class SyntaxGPT(GPT2LMHeadModel):
    def __init__(self, cfg):
        super().__init__(cfg)
        H = cfg.n_embd
        self.combo_embed = nn.Embedding(MAX_COMBOS, H)
        self.merge = nn.Linear(2 * H, H, bias=False)
        self.stored_combo_ids = None

    def prepare_inputs_for_generation(self, input_ids, past_key_values=None, attention_mask=None, **kwargs):
        if past_key_values is None:
            combo_ids = self.stored_combo_ids.to(input_ids.device)
        else:
            combo_ids = input_ids.new_zeros(input_ids.shape)
            attention_mask = None
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "past_key_values": past_key_values,
            "combo_ids": combo_ids,
        }

    def forward(self, input_ids=None, combo_ids=None, attention_mask=None,
                labels=None, inputs_embeds=None, past_key_values=None, **kwargs):
        if inputs_embeds is None:
            if combo_ids is None or combo_ids.size(1) != input_ids.size(1):
                raise ValueError("combo_ids must align with input_ids")
            tok = self.transformer.wte(input_ids)
            cmb = self.combo_embed(combo_ids.to(tok.device))
            inputs_embeds = self.merge(torch.cat([tok, cmb], dim=-1))
        return super().forward(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            labels=labels,
            past_key_values=past_key_values,
            **kwargs,
        )

cfg = GPT2Config.from_pretrained("gpt2", vocab_size=tokenizer.vocab_size)
model = SyntaxGPT(cfg).to(device)
model.config.pad_token_id = tokenizer.pad_token_id

from collections import defaultdict
from sklearn.model_selection import train_test_split

combined_val_docs = []
base_lr = 1e-3
n_epochs = 1
TOTAL = len(lessons)

for idx, (lesson_name, docs) in enumerate(lessons.items(), 1):
    train_docs, val_docs = train_test_split(docs, test_size=0.1, random_state=42)
    combined_val_docs += val_docs

    print(f"\n▶ Lesson {idx}/{TOTAL}: {lesson_name} | "
          f"{len(train_docs)} train | {len(val_docs)} val | epochs={n_epochs} | lr={base_lr:g}")

    train_set = chunkify_docs(train_docs, max_len=64)
    val_set   = chunkify_docs(val_docs, max_len=64)

    args = TrainingArguments(
        output_dir=f"ckpt_{lesson_name}",
        num_train_epochs=n_epochs,
        learning_rate=base_lr,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        eval_strategy="epoch",
        logging_strategy="epoch",
        logging_steps=n_epochs // 5,
        save_strategy="no",
        report_to="none",
        fp16=torch.cuda.is_available(),
        remove_unused_columns=False
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_set,
        eval_dataset=val_set,
        data_collator=collate,
    )
    trainer.train()

# STEP 6 – Final evaluation on combined validation set
final_val = chunkify_docs(combined_val_docs, max_len=64)
metrics = trainer.evaluate(eval_dataset=final_val)

print("\n📊 Combined Val Performance:")
for k, v in metrics.items():
    print(f"{k:20s}: {v:.4f}")


✅ Loaded v3_l1.pkl (61 docs)
✅ Loaded v3_l10.pkl (59 docs)
✅ Loaded v3_l11.pkl (62 docs)
✅ Loaded v3_l12.pkl (71 docs)
✅ Loaded v3_l13.pkl (62 docs)
✅ Loaded v3_l14.pkl (60 docs)
✅ Loaded v3_l15.pkl (61 docs)
✅ Loaded v3_l16.pkl (65 docs)
✅ Loaded v3_l17.pkl (59 docs)
✅ Loaded v3_l18.pkl (50 docs)
✅ Loaded v3_l19.pkl (52 docs)
✅ Loaded v3_l2.pkl (62 docs)
✅ Loaded v3_l20.pkl (66 docs)
✅ Loaded v3_l21.pkl (73 docs)
✅ Loaded v3_l22.pkl (60 docs)
✅ Loaded v3_l23.pkl (64 docs)
✅ Loaded v3_l24.pkl (57 docs)
✅ Loaded v3_l25.pkl (67 docs)
✅ Loaded v3_l26.pkl (49 docs)
✅ Loaded v3_l27.pkl (63 docs)
✅ Loaded v3_l28.pkl (64 docs)
✅ Loaded v3_l29.pkl (60 docs)
✅ Loaded v3_l3.pkl (48 docs)
✅ Loaded v3_l30.pkl (66 docs)
✅ Loaded v3_l31.pkl (64 docs)
✅ Loaded v3_l32.pkl (62 docs)
✅ Loaded v3_l33.pkl (70 docs)
✅ Loaded v3_l34.pkl (60 docs)
✅ Loaded v3_l35.pkl (59 docs)
✅ Loaded v3_l36.pkl (53 docs)
✅ Loaded v3_l37.pkl (74 docs)
✅ Loaded v3_l38.pkl (42 docs)
✅ Loaded v3_l39.pkl (69 docs)
✅ Loaded v3_l

Epoch,Training Loss,Validation Loss
1,8.264000,7.371078



▶ Lesson 2/60: v3_l10.pkl | 53 train | 6 val | epochs=1 | lr=0.001


Epoch,Training Loss,Validation Loss
1,7.597400,7.361845



▶ Lesson 3/60: v3_l11.pkl | 55 train | 7 val | epochs=1 | lr=0.001


...


▶ Lesson 59/60: v3_l8.pkl | 56 train | 7 val | epochs=1 | lr=0.001


Epoch,Training Loss,Validation Loss
1,6.704300,7.352586



▶ Lesson 60/60: v3_l9.pkl | 60 train | 7 val | epochs=1 | lr=0.001


Epoch,Training Loss,Validation Loss
1,6.879200,7.179168



📊 Combined Val Performance:
eval_loss           : 7.3898
eval_runtime        : 1.4099
eval_samples_per_second: 275.1960
eval_steps_per_second: 34.7540
epoch               : 1.0000


In [ ]:
# Assuming `model` and `tokenizer` are already loaded/fine-tuned
save_dir = "my_syntax_gpt_model"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)


('my_syntax_gpt_model/tokenizer_config.json',
 'my_syntax_gpt_model/special_tokens_map.json',
 'my_syntax_gpt_model/vocab.json',
 'my_syntax_gpt_model/merges.txt',
 'my_syntax_gpt_model/added_tokens.json',
 'my_syntax_gpt_model/tokenizer.json')

In [ ]:
save_dir = "my_syntax_gpt_model"

model = SyntaxGPT.from_pretrained(save_dir).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(save_dir)


In [ ]:
# ─────────────────────────────────────────────────────────────
# imports & setup
# ─────────────────────────────────────────────────────────────
import torch
from torch.utils.data import Dataset
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import pickle
from tqdm import tqdm

# assume tokenizer & model already loaded and moved to device:
#   tokenizer.pad_token = tokenizer.eos_token
#   model.to(device)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# ─────────────────────────────────────────────────────────────
# load TREC and Stanza‑parsed docs
# ─────────────────────────────────────────────────────────────
trec = load_dataset("trec")
label_map = trec["train"].features["coarse_label"].names

with open("trec_train_docs.pkl", "rb") as f:
    parsed_train_docs = pickle.load(f)
with open("trec_test_docs.pkl", "rb") as f:
    parsed_test_docs = pickle.load(f)

assert len(parsed_train_docs) == len(trec["train"])
assert len(parsed_test_docs)  == len(trec["test"])

# ─────────────────────────────────────────────────────────────
# encode_doc_with_label: builds input_ids, combo_ids, labels
# ─────────────────────────────────────────────────────────────
def encode_doc_with_label(doc, label_text, max_len=64):
    # prompt tokens
    prompt_text = doc.text.strip()
    prompt_ids  = tokenizer(prompt_text, add_special_tokens=False)["input_ids"]
    # label tokens
    label_ids   = tokenizer.encode(label_text.strip(), add_special_tokens=False)

    # full input = prompt + label
    full_input_ids = prompt_ids + label_ids
    # mask prompt for loss, keep label tokens
    labels = [-100]*len(prompt_ids) + label_ids

    # get combo_ids for prompt only, pad to full length
    _, combo_ids = encode_doc(doc)
    combo_ids = combo_ids[:len(full_input_ids)]
    combo_ids += [0]*(len(full_input_ids) - len(combo_ids))

    # truncate/pad to max_len
    full_input_ids = full_input_ids[:max_len]
    labels         = labels[:max_len]
    combo_ids      = combo_ids[:max_len]

    pad = max_len - len(full_input_ids)
    full_input_ids += [tokenizer.pad_token_id]*pad
    labels         += [-100]*pad
    combo_ids      += [0]*pad

    return {
        "input_ids": full_input_ids,
        "combo_ids": combo_ids,
        "labels":    labels,
    }

# ─────────────────────────────────────────────────────────────
# dataset wrapper
# ─────────────────────────────────────────────────────────────
class TrecSyntaxDataset(Dataset):
    def __init__(self, parsed_docs, label_ids, max_len=64):
        self.samples = []
        for doc, label_id in tqdm(zip(parsed_docs, label_ids), total=len(parsed_docs)):
            sample = encode_doc_with_label(doc, label_map[label_id], max_len)
            self.samples.append(sample)

    def __getitem__(self, idx):
        return self.samples[idx]

    def __len__(self):
        return len(self.samples)

# ─────────────────────────────────────────────────────────────
# build datasets & collator
# ─────────────────────────────────────────────────────────────
train_set = TrecSyntaxDataset(parsed_train_docs, trec["train"]["coarse_label"])
val_set   = TrecSyntaxDataset(parsed_test_docs,  trec["test"]["coarse_label"])

def collate_trec(batch):
    ids   = torch.tensor([b["input_ids"] for b in batch])
    cmb   = torch.tensor([b["combo_ids"] for b in batch])
    labs  = torch.tensor([b["labels"]    for b in batch])
    mask  = (ids != tokenizer.pad_token_id).long()
    return {
        "input_ids":      ids,
        "combo_ids":      cmb,
        "attention_mask": mask,
        "labels":         labs,
    }

# ─────────────────────────────────────────────────────────────
# compute_metrics for Trainer
# ─────────────────────────────────────────────────────────────
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     preds = logits.argmax(-1)
#     acc   = accuracy_score(labels, preds)
#     p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
#     return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

# ─────────────────────────────────────────────────────────────
# Trainer setup & train
# ─────────────────────────────────────────────────────────────
args = TrainingArguments(
    output_dir="syntaxgpt_trec_ckpt",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-4,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="no",
    report_to="none",
    fp16=torch.cuda.is_available(),
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_set,
    eval_dataset=val_set,
    data_collator=collate_trec,
    # compute_metrics=compute_metrics,
)

trainer.train()

# ─────────────────────────────────────────────────────────────
# fixed inference: never feed the true label into generate()
# ─────────────────────────────────────────────────────────────
def predict_label(sample):
    # find where labels start (first non -100)
    prompt_len = next(i for i, x in enumerate(sample["labels"]) if x != -100)

    # slice off just the prompt
    input_ids = torch.tensor([ sample["input_ids"][:prompt_len] ]).to(device)
    combo_ids = torch.tensor([ sample["combo_ids"][:prompt_len] ]).to(device)
    model.stored_combo_ids = combo_ids

    # generate exactly one new token
    print(f"prompt: {tokenizer.decode(input_ids[0], skip_special_tokens=True).strip()}")
    with torch.no_grad():
        out = model.generate(
            input_ids=input_ids,
            attention_mask=(input_ids != tokenizer.pad_token_id).long(),
            max_new_tokens=4
        )[0].tolist()

    # decode only the newly produced token(s)
    new_ids = out[prompt_len:]
    return tokenizer.decode(new_ids, skip_special_tokens=True).strip().upper()

# run through the test set
model.eval()


100%|██████████| 500/500 [00:00<00:00, 1931.13it/s]


Epoch,Training Loss,Validation Loss
1,1.556000,1.223099
2,1.251300,1.073730
3,1.006300,0.873699


SyntaxGPT(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (combo_embed): Embedding(1024, 768)
  (merge): Linear(in_fe

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
import torch.nn.functional as F

correct = 0
total = len(val_set)
all_preds = []
all_golds = []
losses = []

label_name_to_id = {name.upper(): i for i, name in enumerate(label_map)}

for sample, gold_idx in zip(val_set, trec["test"]["coarse_label"]):
    pred = predict_label(sample)
    true = label_map[gold_idx].upper()

    print(f"prompt: {tokenizer.decode(sample['input_ids'], skip_special_tokens=True).strip()}")
    print(f" predict: {pred}\n")

    if pred.startswith(true):  # ✅ soft match
        correct += 1
        pred = true  # overwrite pred to ensure consistency in metrics

    all_preds.append(pred)
    all_golds.append(true)

    # Optional: token-level loss
    try:
        input_ids = torch.tensor([sample["input_ids"]]).to(device)
        labels = torch.tensor([sample["labels"]]).to(device)
        attn_mask = (input_ids != tokenizer.pad_token_id).long()
        model.stored_combo_ids = torch.tensor([sample["combo_ids"]]).to(device)

        with torch.no_grad():
            output = model(input_ids=input_ids, attention_mask=attn_mask, labels=labels)
            losses.append(output.loss.item())
    except Exception:
        pass

# 🎯 Accuracy
acc = correct / total

# 🎯 Classification Metrics
gold_ids = [label_name_to_id.get(label, -1) for label in all_golds]
pred_ids = [label_name_to_id.get(p, -1) for p in all_preds]

filtered_gold_ids = []
filtered_pred_ids = []
for g, p in zip(gold_ids, pred_ids):
    if g != -1 and p != -1:
        filtered_gold_ids.append(g)
        filtered_pred_ids.append(p)

p, r, f1, _ = precision_recall_fscore_support(filtered_gold_ids, filtered_pred_ids, average="macro")

# 📢 Final Results
print(f"\n🔍 Evaluation Results (max_new_tokens=4 + startswith logic):")
print(f"Accuracy:  {acc:.4f}")
print(f"Avg Loss:  {sum(losses)/len(losses):.4f}" if losses else "Loss: (not computed)")
print(f"Recall:    {r:.4f}")
print(f"F1 Score:  {f1:.4f}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt: Here is a question: How far is it from Denver to Aspen ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:
prompt: Here is a question: How far is it from Denver to Aspen ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:NUM
 predict: NUMCCC

prompt: Here is a question: What county is Modesto , California in ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt: Here is a question: What county is Modesto , California in ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:LOC
 predict: DESCCC

prompt: Here is a question: Who was Galileo ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:
prompt: Here is a question: Who was Galileo ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:HUM
 predict: HUMUMUM

prompt: Here is a question: What is an atom ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

prompt: Here is a question: What 's the easiest way to remove wallpaper ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:ENTY
 predict: ENTYYY

prompt: Here is a question: What year did the Titanic start on its journey ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:
prompt: Here is a question: What year did the Titanic start on its journey ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:NUM
 predict: ENTYYY

prompt: Here is a question: How much of an apple is water ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:
prompt: Here is a question: How much of an apple is water ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:NUM
 predict: NUMCCC

prompt: Here is a question: Who was the 22nd President of the US ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt: Here is a question: Who was the 22nd President of the US ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:HUM
 predict: HUMUMUM

prompt: Here is a question: What is the money they use in Zambia ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:
prompt: Here is a question: What is the money they use in Zambia ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:ENTY
 predict: ENTYYY

prompt: Here is a question: How many feet in a mile ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:
prompt: Here is a question: How many feet in a mile ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:NUM
 predict: NUMCCC

prompt: Here is a question: What is the birthstone of October ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt: Here is a question: What is the birthstone of October ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:ENTY
 predict: DESCCC

prompt: Here is a question: What is e-coli ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:
prompt: Here is a question: What is e-coli ? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:DESC
 predict: DESCCC


🔍 Evaluation Results (max_new_tokens=4 + startswith logic):
Accuracy:  0.5320
Loss: (not computed)
Recall:    1.0000
F1 Score:  1.0000
